In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from datetime import date

from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder

import pickle
import random
import time

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
df = pd.read_csv(r'D:\OneDrive - SinoPac\training\month_train_index.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
df['ts'] = pd.to_datetime(df['ts'])
df

,total,capital,index_open_month,index_high_month,index_low_month,index_close,index_return,index_vol_month,industry_return,foreign_ratio,...,pvt,RSI_20,SOk5,SOd3,SOk20,SOd5,industry,ts,StockNo,Y
0,708179000.0,32487.0,8903.77,9807.91,8977.09,9072.57,1.49,163482782.0,4.48,46.91,...,1.260994e+05,0.342664,0.165563,0.276419,0.165563,0.149586,0.0,2007-07-30,1101,1.0
1,753395000.0,32487.0,8983.24,9807.91,8996.81,9287.25,3.24,164062376.0,5.25,46.82,...,1.377989e+06,0.474590,0.443709,0.360064,0.443709,0.247640,0.0,2007-07-31,1101,0.0
2,853873000.0,32487.0,9069.32,9807.91,8891.88,8891.88,-1.95,165296559.0,1.10,46.38,...,1.051719e+06,0.434756,0.086093,0.223078,0.086093,0.193786,0.0,2007-08-01,1101,1.0
3,738356000.0,32487.0,9114.06,9807.91,8727.64,8950.57,-2.17,165395388.0,-0.21,46.08,...,5.036673e+05,0.363721,0.142857,0.182968,0.100559,0.162708,0.0,2007-08-02,1101,1.0
4,722089000.0,32487.0,9169.56,9807.91,8727.64,9057.82,-1.42,163746605.0,0.29,46.14,...,3.798632e+05,0.380502,0.609524,0.396246,0.357542,0.227655,0.0,2007-08-03,1101,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315384,87563000.0,13934.0,10480.67,10619.42,10225.28,10515.51,0.89,108851945.0,-1.66,12.49,...,-8.409451e+05,0.627338,0.848485,0.821769,0.909091,0.811147,20.0,2017-08-25,9945,0.0
315385,221047000.0,13934.0,10396.76,10619.42,10225.28,10525.98,0.95,110362489.0,-1.35,12.34,...,-4.912968e+05,0.700768,0.766667,0.794218,0.888889,0.837061,20.0,2017-08-28,9945,0.0
315386,147118000.0,13934.0,10408.53,10619.42,10225.28,10496.57,0.57,110786512.0,-2.55,12.29,...,-3.718994e+05,0.674077,0.352941,0.573580,0.825397,0.833173,20.0,2017-08-29,9945,0.0
315387,341438000.0,13934.0,10479.26,10619.42,10225.28,10569.40,0.48,111150711.0,-2.19,12.30,...,-1.372293e+05,0.680796,0.647059,0.610319,0.904762,0.857036,20.0,2017-08-30,9945,0.0


In [17]:
test = pd.read_csv(r'D:\OneDrive - SinoPac\training\month_test_index_labeled.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
test['ts'] = pd.to_datetime(test['ts'])
test

,total,capital,index_open_month,index_high_month,index_low_month,index_close,index_return,index_vol_month,industry_return,foreign_ratio,...,pvt,RSI_20,SOk5,SOd3,SOk20,SOd5,industry,ts,StockNo,Y
0,141911000.0,36922.0,10470.96,10619.52,10225.28,10594.82,0.84,112930558.0,0.50,26.48,...,-2.351559e+06,0.561067,0.333333,0.536845,0.565217,0.606908,1.0,2017-09-01,1101,0.0
1,113227000.0,36922.0,10521.69,10619.52,10225.28,10569.87,-0.09,112756731.0,0.11,26.44,...,-2.403120e+06,0.516810,0.100000,0.318422,0.409091,0.540969,1.0,2017-09-04,1101,0.0
2,230732000.0,36922.0,10598.59,10619.52,10225.28,10617.84,0.46,112645149.0,0.16,26.35,...,-2.558614e+06,0.424326,0.133333,0.225878,0.250000,0.443979,1.0,2017-09-05,1101,0.0
3,148040000.0,36922.0,10523.51,10619.52,10225.28,10547.86,0.74,113399199.0,-0.24,26.36,...,-2.602315e+06,0.408180,0.000000,0.112939,0.105263,0.331074,1.0,2017-09-06,1101,0.0
4,141609000.0,36922.0,10267.05,10619.52,10225.28,10609.95,2.71,113334595.0,1.43,26.19,...,-2.272466e+06,0.325931,0.312500,0.184485,0.217391,0.219608,1.0,2017-09-08,1101,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66666,41275000.0,10033.0,10892.25,10927.16,10180.04,10538.11,-3.69,92364219.0,-2.86,16.33,...,4.937152e+03,0.573071,0.666667,0.586815,0.876404,0.856075,20.0,2019-08-23,9945,0.0
66667,54173000.0,10033.0,10898.25,10927.16,10180.04,10354.57,-4.93,92998652.0,-4.42,16.29,...,8.374757e+04,0.489094,0.136364,0.361589,0.730337,0.814162,20.0,2019-08-26,9945,0.0
66668,123244000.0,10033.0,10872.63,10927.16,10180.04,10387.23,-4.58,93245702.0,-4.41,16.15,...,1.132844e+05,0.426322,0.000000,0.180795,0.584270,0.737531,20.0,2019-08-27,9945,1.0
66669,34524000.0,10033.0,10909.98,10835.80,10180.04,10434.29,-3.66,92349525.0,-3.82,16.18,...,1.852686e+05,0.426322,0.125000,0.152897,0.629213,0.701425,20.0,2019-08-28,9945,0.0


In [48]:
X_train = df.drop(columns=['ts','Y', 'StockNo', 'total', 'capital', 'index_open_month', 'index_high_month', 'index_low_month', 'index_close', 'index_vol_month'])
Y_train = df['Y']

In [49]:
X_test = test.drop(columns=['ts','Y', 'StockNo', 'total', 'capital', 'index_open_month', 'index_high_month', 'index_low_month', 'index_close', 'index_vol_month'])
Y_test = test['Y']

In [50]:
X_train

,index_return,industry_return,foreign_ratio,investment_ratio,corporation_ratio,close_return,VWAP_return,index_VWAP_slope,VWAP_day20_return,industry_VWAP_slope,...,industry_pvt,foreign_pvt,investment_pvt,pvt,RSI_20,SOk5,SOd3,SOk20,SOd5,industry
0,1.49,4.48,46.91,1.20,48.41,0.67,0.40,-1.09,6.66,-4.08,...,10993.697784,-19.500000,-2.360000,1.260994e+05,0.342664,0.165563,0.276419,0.165563,0.149586,0.0
1,3.24,5.25,46.82,1.27,48.37,6.55,5.43,2.19,6.65,0.18,...,13823.601908,-217.072193,132.783048,1.377989e+06,0.474590,0.443709,0.360064,0.443709,0.247640,0.0
2,-1.95,1.10,46.38,1.38,48.03,-1.46,1.03,2.98,6.09,-0.07,...,2931.941956,-3.958134,79.008512,1.051719e+06,0.434756,0.086093,0.223078,0.086093,0.193786,0.0
3,-2.17,-0.21,46.08,1.38,47.74,-2.70,-2.07,0.10,6.16,-1.86,...,-432.356106,253.987812,82.008512,5.036673e+05,0.363721,0.142857,0.182968,0.100559,0.162708,0.0
4,-1.42,0.29,46.14,1.38,47.78,-0.65,0.59,2.01,6.09,0.30,...,588.294021,241.737488,81.839900,3.798632e+05,0.380502,0.609524,0.396246,0.357542,0.227655,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315384,0.89,-1.66,12.49,0.00,12.60,1.82,1.67,0.78,-1.73,3.33,...,-790.349958,192.592066,-0.977424,-8.409451e+05,0.627338,0.848485,0.821769,0.909091,0.811147,20.0
315385,0.95,-1.35,12.34,0.00,12.51,3.36,3.07,2.12,-1.16,4.42,...,-710.580530,130.746675,-1.044703,-4.912968e+05,0.700768,0.766667,0.794218,0.888889,0.837061,20.0
315386,0.57,-2.55,12.29,0.00,12.49,1.20,1.75,1.18,-0.79,4.30,...,-1073.597189,129.545536,-1.304650,-3.718994e+05,0.674077,0.352941,0.573580,0.825397,0.833173,20.0
315387,0.48,-2.19,12.30,0.00,12.60,2.11,2.08,1.60,-0.26,4.27,...,-895.733164,148.401041,-1.304650,-1.372293e+05,0.680796,0.647059,0.610319,0.904762,0.857036,20.0


In [51]:
RF2 = RandomForestClassifier(n_estimators=100, 
                            max_depth=20,
                            max_features=0.9,
                            min_samples_leaf=0.1, 
                            min_samples_split=0.1,
                            verbose=1, 
                            n_jobs=-1,
                            oob_score=True)

In [52]:
RF2.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features=0.9, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=0.1, min_samples_split=0.1,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=1,
                       warm_start=False)

In [53]:
RF2.oob_score_

0.5653557987120666

In [54]:
prediction_proba = RF2.predict_proba(X_test)
prediction = RF2.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [38]:
def predict(prediction, threshold):
    predict = []
    for item in prediction:
        if item[1] > threshold:
            predict.append(1)
        else:
            predict.append(0)
            
    return predict

In [61]:
new_prediction = predict(prediction_proba.tolist(), threshold = 0.57)
new_prediction.count(1)/len(new_prediction)

0.18678285911415757

In [56]:
from sklearn.metrics import classification_report
target_names = ['down', 'up']
print(classification_report(Y_test, prediction, target_names=target_names))

              precision    recall  f1-score   support

        down       0.54      0.21      0.30     33146
          up       0.51      0.83      0.63     33525

    accuracy                           0.52     66671
   macro avg       0.53      0.52      0.47     66671
weighted avg       0.53      0.52      0.47     66671



In [57]:
prediction.tolist().count(1)/len(prediction)

0.8087324323918945

In [62]:
from sklearn.metrics import classification_report
target_names = ['down', 'up']
print(classification_report(Y_test, new_prediction, target_names=target_names))

              precision    recall  f1-score   support

        down       0.50      0.82      0.62     33146
          up       0.53      0.20      0.29     33525

    accuracy                           0.51     66671
   macro avg       0.52      0.51      0.46     66671
weighted avg       0.52      0.51      0.45     66671



In [32]:
X_train

,total,capital,index_open,index_high,index_low,index_close,index_return,index_vol,industry_return,foreign_ratio,...,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_20,SOk5,SOd3,SOk20,SOd5,industry
0,12420000.0,4291.0,9037.35,9136.45,8996.81,9072.57,1.49,7332867.0,4.48,1.24,...,5.735849,0.000000e+00,1.026415,-60320.362264,0.258180,0.161290,0.166372,0.113636,0.063144,0.0
1,8958000.0,4291.0,9211.80,9307.81,9173.48,9287.25,3.24,7184071.0,5.25,1.19,...,19.002516,0.000000e+00,1.093082,-107339.028931,0.345591,0.275862,0.221117,0.181818,0.102707,0.0
2,13059000.0,4291.0,9076.11,9080.07,8727.64,8950.57,-2.17,7164273.0,-0.21,1.13,...,45.201946,0.000000e+00,2.102445,-287028.424939,0.272988,0.187500,0.149029,0.063830,0.066922,0.0
3,5181000.0,4291.0,9104.34,9134.60,9025.70,9057.82,-1.42,5865501.0,0.29,1.12,...,45.733311,0.000000e+00,2.191006,-324283.067006,0.272988,0.375000,0.262015,0.127660,0.087169,0.0
4,7151000.0,4291.0,8865.58,8960.75,8856.91,8941.73,-4.57,4658694.0,-0.14,1.12,...,47.003988,0.000000e+00,3.168450,-382538.706104,0.233987,0.105263,0.183639,0.040000,0.071445,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828597,410000.0,1032.0,10497.86,10519.84,10478.72,10515.51,0.89,6024910.0,-1.66,0.30,...,-0.610367,-1.953993e-14,0.297598,-55841.714830,0.550960,0.500000,0.526905,0.458333,0.432271,20.0
1828598,1788000.0,1032.0,10502.80,10542.37,10498.88,10525.98,0.95,5890373.0,-1.35,0.30,...,-0.610367,-1.953993e-14,0.297598,-43915.918540,0.644546,1.000000,0.763452,0.666667,0.510403,20.0
1828599,12537000.0,1032.0,10518.69,10518.69,10463.22,10496.57,0.57,5814380.0,-2.55,0.29,...,-0.862939,-1.953993e-14,0.218176,8841.720164,0.823469,0.566667,0.665060,0.628571,0.549793,20.0
1828600,8826000.0,1032.0,10511.59,10574.53,10511.59,10569.40,0.48,5901288.0,-2.19,0.29,...,-0.862939,-1.953993e-14,0.218176,68666.858542,0.831968,0.718750,0.691905,0.756757,0.618781,20.0


In [21]:
test_df['Y_1_score'] = [item[1] for item in prediction_proba.tolist()]

In [40]:
d = test_df[test_df['corporation_ratio'] < 22]

In [41]:
len(d[(d['Y_1_score'] > 0.5) & (d['Y_weekly'] == 1)])/len(d[(d['Y_1_score'] > 0.5)])

0.6898810908779192

In [46]:
RF2.feature_importances_

array([0.        , 0.0625758 , 0.39851206, 0.11725727, 0.        ,
       0.14391791, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [150]:
df[df.ts.dt.weekday != 4]['ts'].unique()

array(['2007-09-29T00:00:00.000000000', '2008-04-03T00:00:00.000000000',
       '2008-10-09T00:00:00.000000000', '2008-12-31T00:00:00.000000000',
       '2009-01-10T00:00:00.000000000', '2009-01-17T00:00:00.000000000',
       '2009-01-21T00:00:00.000000000', '2009-04-30T00:00:00.000000000',
       '2009-05-27T00:00:00.000000000', '2009-06-06T00:00:00.000000000',
       '2009-08-06T00:00:00.000000000', '2009-12-31T00:00:00.000000000',
       '2010-02-06T00:00:00.000000000', '2010-02-10T00:00:00.000000000',
       '2012-01-18T00:00:00.000000000', '2012-02-04T00:00:00.000000000',
       '2012-03-03T00:00:00.000000000', '2012-12-22T00:00:00.000000000',
       '2013-02-06T00:00:00.000000000', '2013-02-23T00:00:00.000000000',
       '2013-04-03T00:00:00.000000000', '2013-09-14T00:00:00.000000000',
       '2013-09-18T00:00:00.000000000', '2014-01-27T00:00:00.000000000',
       '2014-02-27T00:00:00.000000000', '2014-04-03T00:00:00.000000000',
       '2014-10-09T00:00:00.000000000', '2014-12-27

In [148]:
df.ts

0        2007-08-17
1        2007-08-24
2        2007-08-31
3        2007-09-07
4        2007-09-14
            ...    
286550   2019-08-16
286551   2019-08-23
286552   2019-08-30
286553   2019-09-06
286554   2019-09-12
Name: ts, Length: 286555, dtype: datetime64[ns]